# Retrieve notebook and CPU tables from Baserow

PH, Feb 2025

In [1]:
import requests
import pandas as pd

Baserow API access token (read only access to all tables of the *Computers* project

In [2]:
BASEROW_ACCESS_TOKEN='3dDIsaSBqeszii1EmBJ5FSKVPSJmDms8'

Table IDs

In [3]:
NB_OFFERS_TABLE = '444311'
CPUS_TABLE = '444310'

In [4]:
def get_baserow_rows(TABLE_ID, TOKEN, page=1, size=100):
    """Read rows of table `TABLE_ID`.
    
    Returns (row count, json 'results' field)

    If API error, returns the json error message.
    """
    params = {
        'page': int(page),
        'size': int(size),
    }
    res = requests.get(
        f'https://api.baserow.io/api/database/rows/table/{TABLE_ID}/?user_field_names=true',
        headers={
            'Authorization': f'Token {TOKEN}'
        },
        params = params
    )
    if res.ok:
        json = res.json()
        row_count = json['count']
        results = json['results']
        if len(results) < row_count:
            print(f'Warning: response got paginated ({len(results)} rows retrieved over a total of {row_count}).')
        return row_count, results
    else:
        print(f'Warning: unsuccessul request (code {res.status_code})!')
        return 0, res.json()

## Notebook offers table

In [10]:
row_count, nb_offers_raw = get_baserow_rows(NB_OFFERS_TABLE, BASEROW_ACCESS_TOKEN)
print(f'Notebook offers table has {row_count} rows')
# print first row
nb_offers_raw[0]

Notebook offers table has 22 rows


{'id': 70,
 'order': '0.50000000000000000000',
 'Offer': 'T470S 256GB Good',
 'Active': True,
 'CPU': [{'id': 67,
   'value': 'Intel Core i5-7300U',
   'order': '1.33333333333333325932'}],
 'Condition': {'id': 2750619, 'value': 'Good', 'color': 'darker-blue'},
 'RAM': '16',
 'SSD': '256',
 'Date de création': '2025-02-22',
 'ITJustGood': 'https://www.itjustgood.com/ultrabook-lenovo-thinkpad-t470s-reconditionne-intel-core-i5-7200u-16go-ddr4-256go-ssd-nvme-thunderbolt-3-windows-10-14-full-hd-tactile-0btd8993.html',
 'Price': '190',
 'Screen resolution': {'id': 2750658, 'value': '1920x1080', 'color': 'green'},
 'Thunderbolt': False,
 'Series': [{'id': 69,
   'value': 'ThinkPad T470S',
   'order': '3.50000000000000000000'}],
 'CPU Age': [{'id': 67, 'value': '8.1'}],
 'GB6 Single': [{'id': 67, 'value': '1088'}],
 'GB6 Multi': [{'id': 67, 'value': '2030'}],
 'GB6 Single/€': [{'id': 67, 'value': '5.7'}],
 'GB6 Multi/€': [{'id': 67, 'value': '10.7'}]}

In [11]:
def simplify_nb_offers(table_raw):
    """simplifies the raw notebook offers JSON"""
    table_simple = []
    for row in table_raw:
        row_simple = {
            'Offer': row['Offer'],
            'Price': float(row['Price']),
            'CPU': row['CPU'][0]['value'],
            'CPU Age': float(row['CPU Age'][0]['value']),
            'RAM': float(row['RAM']),
            'SSD': float(row['SSD']),
            'Condition': row['Condition']['value'],
            'Active': row['Active'],
            'GB6 Single': float(row['GB6 Single'][0]['value']),
            'GB6 Multi': float(row['GB6 Multi'][0]['value']),
            'GB6 Single/€': float(row['GB6 Single/€'][0]['value']),
            'GB6 Multi/€': float(row['GB6 Multi/€'][0]['value']),
        }
        table_simple.append(row_simple)
    nrow = len(table_simple)
    print(f'{nrow} notebook offers simplified')
    return table_simple

In [12]:
nb_offers = simplify_nb_offers(nb_offers_raw)
pd.DataFrame.from_dict(nb_offers)

22 notebook offers simplified


,Offer,Price,CPU,CPU Age,RAM,SSD,Condition,Active,GB6 Single,GB6 Multi,GB6 Single/€,GB6 Multi/€
0,T470S 256GB Good,190.0,Intel Core i5-7300U,8.1,16.0,256.0,Good,True,1088.0,2030.0,5.7,10.7
1,Latitude 5480 240 GB Good,210.0,Intel Core i5-6200U,9.7,16.0,240.0,Good,True,916.0,1832.0,4.4,8.7
2,Dell Latitude 5400,250.0,Intel Core i5-8365U,5.9,16.0,240.0,Good,True,1270.0,3167.0,5.1,12.7
3,Dell Latitude 5400,270.0,Intel Core i5-8365U,5.9,16.0,240.0,Very Good,True,1270.0,3167.0,4.7,11.7
4,Dell Latitude 3400,280.0,Intel Core i5-8365U,5.9,16.0,240.0,Very Good,True,1270.0,3167.0,4.5,11.3
5,Lenovo ThinkPad T490S,310.0,Intel Core i5-8365U,5.9,16.0,500.0,Good,True,1270.0,3167.0,4.1,10.2
6,Dell Latitude 5410,330.0,Intel Core i5-10310U,4.9,16.0,240.0,Good,True,1297.0,3450.0,3.9,10.5
7,Dell Latitude 5410,350.0,Intel Core i5-10310U,4.9,16.0,240.0,Good,True,1297.0,3450.0,3.7,9.9
8,Lenovo ThinkPad T490S,360.0,Intel Core i5-8365U,5.9,16.0,500.0,Very Good,True,1270.0,3167.0,3.5,8.8
9,Dell Latitude 5420,360.0,Intel Core i5-10310U,4.9,16.0,500.0,Good,True,1297.0,3450.0,3.6,9.6


## CPUs table

In [13]:
row_count, cpus_raw = get_baserow_rows(CPUS_TABLE, BASEROW_ACCESS_TOKEN)
print(f'CPUs table has {row_count} rows')
# print first row
cpus_raw[0]

CPUs table has 17 rows


{'id': 68,
 'order': '0.50000000000000000000',
 'Name': 'Intel Core i5-6200U',
 'Geekbench 6 Single-Core': '916',
 'Geekbench 6 Multi-Core': '1832',
 'Architecture': [{'id': 1,
   'value': 'Skylake',
   'order': '1.00000000000000000000'}],
 'Launch date': '2015-07-01',
 'Product URL': 'https://www.intel.com/content/www/us/en/products/sku/88193/intel-core-i56200u-processor-3m-cache-up-to-2-80-ghz/specifications.html',
 'Comment': '',
 'Cores': '2',
 'P Cores': None,
 'Notebook offers': [{'id': 71,
   'value': 'Latitude 5480 240 GB Good',
   'order': '0.66666666666666662966'}],
 'NotebookCheck': '',
 'Geekbench 5.5 Single-Core': None,
 'Geekbench 5.5 Multi-Core': None,
 'Age': '9.7',
 'GB6 Source': {'id': 2776699, 'value': 'MyQ50', 'color': 'darker-pink'},
 'GB5 Source': None,
 'Designer': [{'id': 1, 'value': 'Intel'}],
 'Win11 Compatible': False,
 'Notebook series': [{'ids': {'database_table_444311': 71,
    'database_table_444404': 70},
   'value': 'Latitude 5480'}]}

In [15]:
def simplify_cpus(table_raw):
    """simplifies the raw notebook offers JSON"""
    table_simple = []
    for row in table_raw:
        row_simple = {
            'Name': row['Name'],
            'Launch date': row['Launch date'],
            'Cores': row['Cores'],
            'Age': row['Age'],
            'Architecture': row['Architecture'][0]['value'],
            'GB6 Single': float(row['Geekbench 6 Single-Core']),
            'GB6 Multi': float(row['Geekbench 6 Multi-Core']),
            'Designer': row['Designer'][0]['value'],
            'Win11': 'Yes' if row['Win11 Compatible'] else 'No',
            
        }
        table_simple.append(row_simple)
    nrow = len(table_simple)
    print(f'{nrow} cpus simplified')
    return table_simple

In [16]:
cpus = simplify_cpus(cpus_raw)
cpus_df = pd.DataFrame.from_dict(cpus)
#cpus_df['Launch date'] = pd.to_datetime(cpus_df['Launch date']) # date string to datetime. useless to make a CSV
cpus_df

17 cpus simplified


,Name,Launch date,Cores,Age,Architecture,GB6 Single,GB6 Multi,Designer,Win11
0,Intel Core i5-6200U,2015-07-01,2,9.7,Skylake,916.0,1832.0,Intel,No
1,Intel Core i5-6300U,2015-07-01,2,9.7,Skylake,965.0,1910.0,Intel,No
2,Intel Core i5-7300U,2017-01-01,2,8.1,Kaby Lake,1088.0,2030.0,Intel,No
3,Intel Core i5-8250U,2017-07-01,4,7.6,Kaby Lake R,1145.0,3126.0,Intel,Yes
4,Intel Core i5-8350U,2017-07-01,4,7.6,Kaby Lake R,1191.0,3237.0,Intel,Yes
5,Intel Core i5-8365U,2019-04-01,4,5.9,Whiskey Lake,1270.0,3167.0,Intel,Yes
6,Intel Core i5-9400H,2019-04-01,4,5.9,Coffee Lake,1429.0,4347.0,Intel,Yes
7,Intel Core i5-10210U,2019-07-01,4,5.7,Comet Lake,1229.0,3088.0,Intel,Yes
8,Intel Core i5-10310U,2020-04-01,4,4.9,Comet Lake,1297.0,3450.0,Intel,Yes
9,Intel Core i5-1135G7,2020-07-01,4,4.6,Tiger Lake,1702.0,4694.0,Intel,Yes


In [17]:
cpus_df.to_csv('CPUs.csv', index=False)